# OBIETTIVO: Reynolds number comparison analysis

This notebook compares HOSVD core tensors across different Reynolds numbers: Re = [7000, 8000, 9000, 10000]

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorly as tl
from tensorly.decomposition import tucker
from tensorly.tucker_tensor import tucker_to_tensor
from tensorly.tenalg import mode_dot, multi_mode_dot
import kagglehub
import h5py
from tqdm import tqdm
import pandas as pd
import kagglehub
import global_variables

## Data Loading
Load data for all Reynolds numbers: 7000, 8000, 9000, 10000

In [ ]:
n_snapshots = 200
subsample_x = 10
subsample_y = 10

# Reynolds numbers to compare
reynolds_numbers = [7000, 8000, 9000, 10000]
paths = [f"sharmapushan/hydrogen-jet-{re}" for re in reynolds_numbers]

data_paths = [kagglehub.dataset_download(name) for name in paths]

# Get metadata from first dataset (assuming same grid for all Re)
with open(data_paths[0] + '/info.json') as f:
    metadata = json.load(f)
    
Nx, Ny = metadata['global']['Nxyz']
Nx_sub = Nx // subsample_x
Ny_sub = Ny // subsample_y

component_names = global_variables.component_names
n_species = global_variables.n_species
molar_masses = global_variables.molar_masses
file_key_map = global_variables.file_key_map
Lx, Ly = global_variables.Lx, global_variables.Ly

print(f"Grid dimensions: Nx={Nx}, Ny={Ny}")
print(f"Subsampled grid: Nx_sub={Nx_sub}, Ny_sub={Ny_sub}")
print(f"Reynolds numbers: {reynolds_numbers}")

In [ ]:
tensors = {}
for data_path, re_num in zip(data_paths, reynolds_numbers):
    key = f"Re{re_num}"
    print(f"\nLoading data for {key}...")
    
    # Load metadata for this dataset
    with open(data_path + '/info.json') as f:
        metadata_re = json.load(f)
    
    tensor = np.zeros((Ny//subsample_y, Nx//subsample_x, n_species, n_snapshots))
    
    for t_idx in tqdm(range(n_snapshots), desc=f"Loading {key} snapshots"):
        for new_idx, (comp_name, orig_idx) in enumerate(zip(component_names, range(n_species))):
            filename_key = file_key_map[comp_name]
            filename = metadata_re['local'][t_idx][filename_key]
            data = np.fromfile(f"{data_path}/{filename}", dtype='<f4').reshape(Ny, Nx)
            molar_data = data / molar_masses[comp_name]
            tensor[:, :, new_idx, t_idx] = molar_data[::subsample_x, ::subsample_y]
    
    tensors[key] = tensor
    print(f"{key} tensor shape: {tensor.shape}")

## Tensor Scaling and Centering
Apply log-scaling and standardization to all datasets

In [ ]:
def scale_and_center_tensors(tensors, component_names, log_scale=True, 
                             temporal_m=False, std_scale=True, epsilon=1e-12):
    tensors_scaled = {}
    
    for dataset_path, tensor in tensors.items():
        tensor_scaled = tensor.copy()  # Shape: (x, y, species, t)        
        for c_idx, comp_name in enumerate(component_names):
            component_data = tensor_scaled[:, :, c_idx, :].copy()            
            if log_scale:
                component_data = np.log10(np.maximum(component_data, epsilon))
            if temporal_m:
                temporal_mean = component_data.mean(axis=-1, keepdims=True)  # Mean over time
                component_data = component_data - temporal_mean
            if std_scale:
                mean_val = component_data.mean()  # Should be ~0 if temporal_m=True
                std_val = component_data.std()                
                if std_val < epsilon:
                    std_val = epsilon  # Prevent divide-by-zero
                
                component_data = (component_data - mean_val) / std_val
            
            # Store processed component
            tensor_scaled[:, :, c_idx, :] = component_data
        
        tensors_scaled[dataset_path] = tensor_scaled
    
    return tensors_scaled

In [ ]:
# Apply standard scaling with log-transform
tensors_scaled = scale_and_center_tensors(tensors, component_names, log_scale=True, 
                                          temporal_m=False, std_scale=True)

print("Scaled tensors:")
for key in tensors_scaled.keys():
    print(f"  {key}: {tensors_scaled[key].shape}")

## HOSVD Decomposition
Perform Higher-Order Singular Value Decomposition for all Reynolds numbers

In [ ]:
decomposition_results = {}  # store factors and cores per dataset

for dataset_key, tensor in tensors_scaled.items():
    print("\n" + "=" * 100)
    print(f"Performing HOSVD for: {dataset_key}")
    print("=" * 100)
    print(f"Tensor shape: (Ny={tensor.shape[0]}, Nx={tensor.shape[1]}, n_chem={tensor.shape[2]}, n_time={tensor.shape[3]})")
    
    # Compute factor matrices for each mode
    print("Computing U_y (mode 0: spatial Y)...")
    U_y, _, _ = np.linalg.svd(tl.unfold(tensor, mode=0), full_matrices=False)
    
    print("Computing U_x (mode 1: spatial X)...")
    U_x, _, _ = np.linalg.svd(tl.unfold(tensor, mode=1), full_matrices=False)
    
    print("Computing U_chem (mode 2: chemical)...")
    U_chem, _, _ = np.linalg.svd(tl.unfold(tensor, mode=2), full_matrices=False)
    
    print("Computing U_time (mode 3: time)...")
    U_time, _, _ = np.linalg.svd(tl.unfold(tensor, mode=3), full_matrices=False)
    
    # Compute core tensor
    print("Computing core tensor...")
    core = multi_mode_dot(tensor, [U_y.T, U_x.T, U_chem.T, U_time.T], modes=[0, 1, 2, 3])
    
    decomposition_results[dataset_key] = {
        "core": core,
        "U_y": U_y,
        "U_x": U_x,
        "U_chem": U_chem,
        "U_time": U_time,
        "factors": [U_y, U_x, U_chem, U_time],
    }
    
    print(f"Core tensor shape: {core.shape}")
    print(f"Core tensor norm: {np.linalg.norm(core):.4f}")

print("\n" + "=" * 100)
print("HOSVD decomposition completed for all Reynolds numbers")
print("=" * 100)

## Core Tensor Singular Values Comparison
Compare core tensor structure across different Reynolds numbers

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = plt.cm.viridis(np.linspace(0, 1, len(reynolds_numbers)))

for idx, (re_num, color) in enumerate(zip(reynolds_numbers, colors)):
    key = f"Re{re_num}"
    core = decomposition_results[key]['core']
    label = f"Re = {re_num}"
    
    # Spatial Y dimension (U_y)
    sv_y = [np.linalg.norm(core[i, :, :, :]) for i in range(core.shape[0])]
    ax = axes[0, 0]
    ax.scatter(range(len(sv_y)), sv_y, color=color, s=15, alpha=0.7, label=label)
    ax.plot(range(len(sv_y)), sv_y, color=color, alpha=0.3, linewidth=1.5)
    
    # Spatial X dimension (U_x)
    sv_x = [np.linalg.norm(core[:, i, :, :]) for i in range(core.shape[1])]
    ax = axes[0, 1]
    ax.scatter(range(len(sv_x)), sv_x, color=color, s=15, alpha=0.7, label=label)
    ax.plot(range(len(sv_x)), sv_x, color=color, alpha=0.3, linewidth=1.5)
    
    # Chemical dimension (U_chem)
    sv_chem = [np.linalg.norm(core[:, :, i, :]) for i in range(core.shape[2])]
    ax = axes[1, 0]
    ax.scatter(range(len(sv_chem)), sv_chem, color=color, s=15, alpha=0.7, label=label)
    ax.plot(range(len(sv_chem)), sv_chem, color=color, alpha=0.3, linewidth=1.5)
    
    # Time dimension (U_time)
    sv_time = [np.linalg.norm(core[:, :, :, i]) for i in range(core.shape[3])]
    ax = axes[1, 1]
    ax.scatter(range(len(sv_time)), sv_time, color=color, s=15, alpha=0.7, label=label)
    ax.plot(range(len(sv_time)), sv_time, color=color, alpha=0.3, linewidth=1.5)

# Spatial Y
ax = axes[0, 0]
ax.set_xlabel('U_y Mode Index', fontsize=12)
ax.set_ylabel('Core Singular Value (Frobenius norm)', fontsize=12)
ax.set_title('Core Singular Values - U_y (Spatial Y)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
ax.legend(fontsize=10, loc='best')

# Spatial X
ax = axes[0, 1]
ax.set_xlabel('U_x Mode Index', fontsize=12)
ax.set_ylabel('Core Singular Value (Frobenius norm)', fontsize=12)
ax.set_title('Core Singular Values - U_x (Spatial X)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
ax.legend(fontsize=10, loc='best')

# Chemical
ax = axes[1, 0]
ax.set_xlabel('U_chem Mode Index', fontsize=12)
ax.set_ylabel('Core Singular Value (Frobenius norm)', fontsize=12)
ax.set_title('Core Singular Values - U_chem (Chemical)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
ax.legend(fontsize=10, loc='best')

# Time
ax = axes[1, 1]
ax.set_xlabel('U_time Mode Index', fontsize=12)
ax.set_ylabel('Core Singular Value (Frobenius norm)', fontsize=12)
ax.set_title('Core Singular Values - U_time (Time)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
ax.legend(fontsize=10, loc='best')

fig.suptitle('Core Tensor Singular Values: Reynolds Number Comparison', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Chemical Factor Loadings (U_chem)
Visualize how chemical species contribute to each mode for different Re

In [ ]:
# Visualize Chemical Loadings with Signs for all Reynolds numbers
n_re = len(reynolds_numbers)
fig, axes = plt.subplots(1, n_re, figsize=(5*n_re, 6))

if n_re == 1:
    axes = [axes]

for idx, re_num in enumerate(reynolds_numbers):
    key = f"Re{re_num}"
    U_chem = decomposition_results[key]['U_chem']  # shape: (n_species, n_species)
    
    ax = axes[idx]
    im = ax.imshow(U_chem.T, cmap='RdBu_r', aspect='auto', 
                    vmin=-np.abs(U_chem).max(), 
                    vmax=np.abs(U_chem).max())
    ax.set_xlabel('Species Index', fontsize=12)
    ax.set_ylabel('Chemical Mode Index', fontsize=12)
    ax.set_title(f'Re = {re_num}', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(component_names)))
    ax.set_xticklabels(component_names, rotation=45, ha='right')
    plt.colorbar(im, ax=ax, label='U_chem Loading Value')

fig.suptitle('U_chem: Chemical Factor Loadings Across Reynolds Numbers', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Detailed Chemical Mode Comparison
Bar plots showing first few chemical modes for each Reynolds number

In [ ]:
# Visualize first few chemical modes as bar plots
n_modes_to_show = min(4, len(component_names))

for re_num in reynolds_numbers:
    key = f"Re{re_num}"
    U_chem = decomposition_results[key]['U_chem']
    
    fig, axes = plt.subplots(1, n_modes_to_show, figsize=(20, 5))
    
    x = np.arange(len(component_names))
    
    for mode_idx in range(n_modes_to_show):
        ax = axes[mode_idx] if n_modes_to_show > 1 else axes
        values = U_chem[:, mode_idx]
        colors = ['red' if v < 0 else 'blue' for v in values]
        ax.bar(x, values, color=colors, alpha=0.7)
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        ax.set_xlabel('Species', fontsize=11)
        ax.set_ylabel('U_chem Loading Value', fontsize=11)
        ax.set_title(f'U_chem Mode {mode_idx}', fontsize=12, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(component_names, rotation=45, ha='right', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    fig.suptitle(f'Re = {re_num}: First {n_modes_to_show} U_chem Modes', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## Spatial Mode Coupling Analysis
Examine how spatial modes (U_y and U_x) couple through the core tensor

In [ ]:
# Spatial coupling: sum over chemical and time dimensions
sum_axes = (2, 3)  # Chemical and time dimensions
n_y_modes = 15
n_x_modes = 15

fig, axes = plt.subplots(1, len(reynolds_numbers), figsize=(5*len(reynolds_numbers), 5))

if len(reynolds_numbers) == 1:
    axes = [axes]

for idx, re_num in enumerate(reynolds_numbers):
    key = f"Re{re_num}"
    core = decomposition_results[key]['core']
    
    # Sum over chemical and time dimensions
    spatial_coupling = np.sum(core, axis=sum_axes)[:n_y_modes, :n_x_modes]
    
    ax = axes[idx]
    im = ax.imshow(
        spatial_coupling,
        cmap='RdBu_r',
        aspect='auto',
        vmin=-np.abs(spatial_coupling).max(),
        vmax=np.abs(spatial_coupling).max()
    )
    
    ax.set_xlabel('U_x Mode Index', fontsize=12)
    ax.set_ylabel('U_y Mode Index', fontsize=12)
    ax.set_title(f"Re = {re_num}", fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Core interaction strength')

fig.suptitle(f"Spatial Mode Coupling (summed over chemical & time)", 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Pairwise Reynolds Number Comparison
Compare spatial coupling between consecutive Reynolds numbers

In [ ]:
# Pairwise comparison between consecutive Reynolds numbers
for i in range(len(reynolds_numbers) - 1):
    re1 = reynolds_numbers[i]
    re2 = reynolds_numbers[i + 1]
    
    key1 = f"Re{re1}"
    key2 = f"Re{re2}"
    
    core1 = decomposition_results[key1]['core']
    core2 = decomposition_results[key2]['core']
    
    # Compute spatial coupling
    coupling1 = np.sum(core1, axis=(2, 3))[:n_y_modes, :n_x_modes]
    coupling2 = np.sum(core2, axis=(2, 3))[:n_y_modes, :n_x_modes]
    
    # Normalize for comparison
    coupling1_abs = np.abs(coupling1)
    coupling2_abs = np.abs(coupling2)
    
    norm1 = np.linalg.norm(coupling1_abs, 'fro')
    norm2 = np.linalg.norm(coupling2_abs, 'fro')
    
    # Create comparison plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # First Re
    im0 = axes[0].imshow(coupling1_abs / norm1, cmap='Reds')
    axes[0].set_xlabel('U_x Mode Index')
    axes[0].set_ylabel('U_y Mode Index')
    axes[0].set_title(f"Re = {re1} (normalized)")
    plt.colorbar(im0, ax=axes[0])
    
    # Second Re
    im1 = axes[1].imshow(coupling2_abs / norm2, cmap='Reds')
    axes[1].set_xlabel('U_x Mode Index')
    axes[1].set_ylabel('U_y Mode Index')
    axes[1].set_title(f"Re = {re2} (normalized)")
    plt.colorbar(im1, ax=axes[1])
    
    # Difference
    diff = (coupling1_abs / norm1) - (coupling2_abs / norm2)
    im2 = axes[2].imshow(diff, cmap='coolwarm', 
                         vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)))
    axes[2].set_xlabel('U_x Mode Index')
    axes[2].set_ylabel('U_y Mode Index')
    axes[2].set_title(f"Difference (Re{re1} - Re{re2})")
    plt.colorbar(im2, ax=axes[2])
    
    fig.suptitle(f"Spatial Coupling Comparison: Re = {re1} vs Re = {re2}", 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## Energy Distribution Analysis
Analyze how energy is distributed across modes for different Reynolds numbers

In [ ]:
# Energy distribution across modes
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for idx, re_num in enumerate(reynolds_numbers):
    key = f"Re{re_num}"
    core = decomposition_results[key]['core']
    
    # Compute cumulative energy for each mode
    sv_y = np.array([np.linalg.norm(core[i, :, :, :]) for i in range(core.shape[0])])
    sv_x = np.array([np.linalg.norm(core[:, i, :, :]) for i in range(core.shape[1])])
    sv_chem = np.array([np.linalg.norm(core[:, :, i, :]) for i in range(core.shape[2])])
    sv_time = np.array([np.linalg.norm(core[:, :, :, i]) for i in range(core.shape[3])])
    
    # Normalize to get energy fractions
    energy_y = (sv_y**2) / (sv_y**2).sum()
    energy_x = (sv_x**2) / (sv_x**2).sum()
    energy_chem = (sv_chem**2) / (sv_chem**2).sum()
    energy_time = (sv_time**2) / (sv_time**2).sum()
    
    # Cumulative energy
    cum_energy_y = np.cumsum(energy_y)
    cum_energy_x = np.cumsum(energy_x)
    cum_energy_chem = np.cumsum(energy_chem)
    cum_energy_time = np.cumsum(energy_time)
    
    color = colors[idx]
    label = f"Re = {re_num}"
    
    axes[0, 0].plot(cum_energy_y, color=color, label=label, linewidth=2)
    axes[0, 1].plot(cum_energy_x, color=color, label=label, linewidth=2)
    axes[1, 0].plot(cum_energy_chem, color=color, label=label, linewidth=2)
    axes[1, 1].plot(cum_energy_time, color=color, label=label, linewidth=2)

# Format plots
axes[0, 0].set_xlabel('U_y Mode Index')
axes[0, 0].set_ylabel('Cumulative Energy Fraction')
axes[0, 0].set_title('Spatial Y Dimension')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()
axes[0, 0].set_ylim([0, 1.05])

axes[0, 1].set_xlabel('U_x Mode Index')
axes[0, 1].set_ylabel('Cumulative Energy Fraction')
axes[0, 1].set_title('Spatial X Dimension')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()
axes[0, 1].set_ylim([0, 1.05])

axes[1, 0].set_xlabel('U_chem Mode Index')
axes[1, 0].set_ylabel('Cumulative Energy Fraction')
axes[1, 0].set_title('Chemical Dimension')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()
axes[1, 0].set_ylim([0, 1.05])

axes[1, 1].set_xlabel('U_time Mode Index')
axes[1, 1].set_ylabel('Cumulative Energy Fraction')
axes[1, 1].set_title('Time Dimension')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()
axes[1, 1].set_ylim([0, 1.05])

fig.suptitle('Cumulative Energy Distribution Across Reynolds Numbers', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()